# Assignment: Analyzing Airline Flight Delays 
#### By Brett Hallum, Chris Ficklin, and Ryan Shuhart<br>April 2017

For a full treatment of the unit 14 case study, please review module 14.3. Some points from the video are given below.

Work with the airline data set (use R or Python to manage out-of-core).
Answer the following questions by using the split-apply-combine technique:
* Which airports are most likely to be delayed flying out of or into?
* Which flights with same origin and destination are most likely to be delayed?
* Can you regress how delayed a flight will be before it is delayed?
* What are the most important features for this regression?

Remember to properly cross-validate models.

Use meaningful evaluation criteria.

Create at least one new feature variable for the regression.

In [53]:
import dask.dataframe as dd #http://dask.pydata.org/en/latest/
import pandas as pd
from datetime import datetime
from bokeh.io import output_notebook

from dask.distributed import Client
client = Client(set_as_default=True)

### Other Settings
# Show more rows
pd.options.display.max_rows = 999

# Prevent scientific notation of decimals
pd.set_option('precision',3)
pd.options.display.float_format = '{:,.3f}'.format

In [54]:
# Allow inline display of bokeh graphics
output_notebook()

Loading BokehJS ...

## [Here is some info about Dask]...

...General facts about Dask... blah blah

#### Comparison of Dask Files
* Ryan's Hardware: 
    - CPU: Intel i5-4300M @ 2.60GHz
    - Disk: Samsung SSD 850 Pro
    - RAM: 8 GB
    

* Dask using original csv:
    - no conversion
    - size on disk
        - 11.2 gb
    - benchmark of describing 'Distance':
        - Approx. 4 minutes
* Dask using uncompressed parquet: 
    - conversion to parquet
        - approx 10 minutes
    - size on disk:
        - 13.8 gb
    - benchmark of describing 'Distance':
        - 1 loop, best of 3: 6.2 s per loop
* Dask using gzip compressed parquet:
    - converstion to parquet
        - approx 42 minutes
    - size on disk:
        - 1.36 gb <- big difference
    - benchmark of describing 'Distance':
        - 1 loop, best of 3: 8.83 s per loop

#### Summary
Dask allows for out of core management of data sets. CSV files are universal, but slow to process. Converting to parquet file format, speeds up the process by a factor of 38. Using the gzip compression, reduces size on disk from 13.8gb to 1.36 or about 10% of the uncompressed size. This comes in handy for a distributed processing in a cluster since not as much network bandwidth would be needed. The trade off of compression is a 42.4% increasing in processing time, however, 3 additional seconds is hardly noticable, but might be more of an issue for other tasks. 

## Data

In [55]:
# http://stat-computing.org/dataexpo/2009/the-data.html
var_desc = pd.read_csv("../ref/var_descriptions.csv", index_col='var_id')
var_desc

,Name,Data Type,Description
var_id,,,
1,Year,int64,1987-2008
2,Month,int64,1 - 12
3,DayofMonth,int64,1 - 31
4,DayOfWeek,int64,1 (Monday) - 7 (Sunday)
5,DepTime,float64,"actual departure time (local, hhmm)"
6,CRSDepTime,int64,"scheduled departure time (local, hhmm)"
7,ArrTime,float64,"actual arrival time (local, hhmm)"
8,CRSArrTime,int64,"scheduled arrival time (local, hhmm)"
9,UniqueCarrier,O,unique carrier code


In [56]:
# Data Location
# Ryan's
parq_folder = "C:/Users/ryan.shuhart/Downloads/AirlineDelays.tar/AirlineDelays/parquet_small/"
#parq_folder = "C:/Users/ryan.shuhart/Downloads/AirlineDelays.tar/AirlineDelays/parquet/"

# Columns for Analysis
columns = ['Origin','Year', 'CRSDepTime','DepDelay','DayOfWeek', 'DepTime', 
           'Month', 'DepTime', 'UniqueCarrier', 'Dest']

# Load compressed Parquet format of all years ~2 sec
start = datetime.now()
df = dd.read_parquet(parq_folder)#, columns=columns)
print("Load parquet time: ", datetime.now() - start)
print()

# Length of dask dataframe ~3 min
start = datetime.now()
print("There are {:,d} rows".format(len(df))) #123,534,969 Matches Eric Larson
print("Time to determine row count: ", datetime.now() - start)

Load parquet time:  0:00:01.678816

There are 12,353,502 rows
Time to determine row count:  0:00:22.305544


### Glance at Beginning and End

In [57]:
print("First 5 rows:")
df.head()

First 5 rows:


,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,1987,10,8,4,"1,038.000",1040,"1,202.000",1200,NW,1080,...,nan,nan,0,None,0,nan,nan,nan,nan,nan
1,1987,10,3,6,907.000,910,"1,011.000",1020,PI,826,...,nan,nan,0,None,0,nan,nan,nan,nan,nan
2,1987,11,30,1,"1,113.000",1105,"1,324.000",1308,PI,33,...,nan,nan,0,None,0,nan,nan,nan,nan,nan
3,1987,10,2,5,"1,705.000",1705,"1,757.000",1752,NW,1286,...,nan,nan,0,None,0,nan,nan,nan,nan,nan
4,1987,10,7,3,810.000,810,900.000,900,WN,226,...,nan,nan,0,None,0,nan,nan,nan,nan,nan


In [58]:
print("Last 5 rows:")
df.tail()

Last 5 rows:


,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
49917,2008,12,2,2,"2,140.000",2155,"2,210.000",2229,HA,395,...,5.000,5.000,0,None,0,nan,nan,nan,nan,nan
49918,2008,12,8,1,"1,608.000",1559,"1,816.000",1802,9E,2942,...,13.000,20.000,0,None,0,nan,nan,nan,nan,nan
49919,2008,12,7,7,557.000,600,840.000,833,AS,244,...,7.000,23.000,0,None,0,nan,nan,nan,nan,nan
49920,2008,12,17,3,"1,025.000",1015,"1,400.000",1400,CO,1544,...,5.000,21.000,0,None,0,nan,nan,nan,nan,nan
49921,2008,12,10,3,"1,801.000",1810,"2,042.000",2059,F9,788,...,6.000,13.000,0,None,0,nan,nan,nan,nan,nan


## Feature Creation

In [60]:
# Create an hour field
# 2400 minutes from midnight reduced to 2399 then int division drops to 23
df = df.assign(Hour=df.CRSDepTime.clip(upper=2399)//100) 

## Flight Delays

When a schedule airflight is behind more than 15 minutes then it is officially delayed. Same logic will be followed for arrival times. Only arrivals 15 minutes past scheduled time will be considered late

http://aspmhelp.faa.gov/index.php/Types_of_Delay

### Aggregations

In [65]:
import dask
start = datetime.now()
# Define some aggregations to plot
aggregations = (
    #1 Average departure delay by year
    df.groupby('Year').DepDelay.mean(),
    
    #2 Average departure delay by Month
    df.groupby('Month').DepDelay.mean(), 
    
    #3 Average departure delay by hour of day
    df.groupby('Hour').DepDelay.mean(), 
    
    #4 Average departure delay by Carrier, top 15
    df.groupby('UniqueCarrier').DepDelay.mean().nlargest(15), 
    
    #5 Average arrival delay by destination, top 15
    (df.groupby('Dest').ArrDelay.mean().nlargest(15) 
     .reset_index().rename(columns={'ArrDelay':'AvgArrDelay'})),
    
    #6 Count of arrivals to destinations, excludes missing
    (df.groupby('Dest').ArrDelay.count() 
     .reset_index().rename(columns={'ArrDelay':'ArrCount'})),
    
    #7 Average departure delay by origin, top 15
    (df.groupby('Origin').DepDelay.mean().nlargest(15).reset_index().rename(columns={'DepDelay':'AvgDepDelay'})),
    
    #8 Count of departures by origin, excludes missing
    (df.groupby('Origin').DepDelay.count().reset_index().rename(columns={'DepDelay':'DepCount'})), 
    
    #9 Average departure by origin and destination
    (df.groupby(['Origin','Dest']).DepDelay.mean().reset_index().rename(columns={'DepDelay':'AvgDepDelay'})),
    
    #10 Count of departures between origin and destination
    (df.groupby(['Origin','Dest']).DepDelay.count().reset_index().rename(columns={'DepDelay':'DepCount'})),
    
    #11 Percentage of officially delayed flights by origin
    ((df[df.DepDelay>15].groupby('Origin').DepDelay.count() / df.groupby('Origin').DepDelay.count())
     .reset_index().rename(columns={'DepDelay':'PercDepDelay'})),
    
    #12 Percentage of officially late flights by destination
    ((df[df.ArrDelay>15].groupby('Dest').ArrDelay.count() / df.groupby('Dest').ArrDelay.count())
     .reset_index().rename(columns={'ArrDelay':'PercArrDelay'})),
                
    #13 Percentage of officially delayed flights by origin and destination
    ((df[df.DepDelay>15].groupby(['Origin','Dest']).DepDelay.count() / df.groupby(['Origin','Dest']).DepDelay.count())
     .reset_index().rename(columns={'DepDelay':'PercDepDelay'})),
                
    #14 Percentage of officially late flights by origin and destination
    ((df[df.ArrDelay>15].groupby(['Origin','Dest']).ArrDelay.count() / df.groupby(['Origin','Dest']).ArrDelay.count())
     .reset_index().rename(columns={'ArrDelay':'PercArrDelay'}))
)

# Compute them all in a single pass over the data
(delayed_by_year, #1
delayed_by_month, #2
delayed_by_hour, #3
delayed_by_carrier, #4
delayed_by_dest, #5
delayed_by_dest_count, #6
delayed_by_origin, #7
delayed_by_origin_count, #8
delayed_by_origin_dest, #9
delayed_by_origin_dest_count, #10
pct_delayed_by_origin, #11
pct_late_by_dest, #12
pct_delayed_by_origin_dest, #13
pct_late_by_origin_dest #14
) = dask.compute(*aggregations)
print(datetime.now() - start)

0:04:20.379300


### Visualization of Average Delay

In [52]:
from bokeh.plotting import figure, show
from bokeh.charts.attributes import cat
from bokeh.charts import Bar
from bokeh.layouts import gridplot

# Average Delay by Year
p1 = Bar(delayed_by_year.reset_index(), 'Year', values= 'DepDelay', 
         legend=False, ylabel="Average Delay in Minutes", 
         title="Average Delay by Year")

# Average Delay by Month
delayed_by_month = delayed_by_month.sort_index()
p2 = Bar(delayed_by_month.reset_index(), 'Month', values= 'DepDelay', 
         legend=False, ylabel="Average Delay in Minutes", 
         title="Average Delay by Month")

# Average Delay by Hour of Day
p3 = Bar(delayed_by_hour.reset_index(), 'Hour', values= 'DepDelay', 
         legend=False, ylabel="Average Delay in Minutes",
         title="Average Delay by Hour of Day")

# Average Delay by Carrier
p4 = Bar(delayed_by_carrier.reset_index(), label=cat('Unique Carrier', sort=False), values= 'DepDelay', 
         legend=False, ylabel="Average Delay in Minutes", title="Average Delay by Carrier")

show(gridplot([[p1,p2],[p3,p4]], plot_width=400, plot_height=300))

TypeError: 'NoneType' object is not iterable

## Which airports are most likely to be delayed flying out of or into?

<font color="red">The answer could be found averaging the the average departure delay with the average arrival delay </font>

In [19]:
# Average Delay by Origin
p5 = Bar(delayed_by_origin, label=cat('Origin', sort=False), values= 'AvgDepDelay', 
         legend=False, ylabel="Average Delay in Minutes", title="Average Delay by Origin")

# Average Delay by Destination
p6 = Bar(delayed_by_dest, label=cat('Dest', sort=False), values= 'AvgArrDelay', 
         legend=False, ylabel="Average Delay in Minutes", title="Average Delay by Destination")

show(gridplot([[p5, p6]], plot_width=400, plot_height=300)) 

In [ ]:
# Average Delay by Origin
p7 = Bar(delayed_by_origin, label=cat('Origin', sort=False), values= 'AvgDepDelay', 
         legend=False, ylabel="Average Delay in Minutes", title="Average Departure Delay by Origin")

# Average Delay by Destination
p8 = Bar(delayed_by_dest, label=cat('Dest', sort=False), values= 'AvgArrDelay', 
         legend=False, ylabel="Average Delay in Minutes", title="Average Arrival Delay by Destination")

show(gridplot([[p5, p6]], plot_width=400, plot_height=300)) 

In [18]:
delayed_by_dest

,Dest,AvgArrDelay
0,PVU,28.000
1,OTH,26.792
2,SOP,25.599
3,HHH,23.731
4,MQT,23.216
5,ACK,22.587
6,YAP,16.894
7,MCN,16.212
8,LMT,16.120
9,MAZ,14.690


## Which flights with same origin and destination are most likely to be delayed?

In [51]:
org_dest_pcts = (pd.merge(pct_delayed_by_origin_dest, pct_late_by_origin_dest, on=['Origin','Dest'])
                 .assign(AvgDelay= lambda x: (x['PercDepDelay'] + x['PercArrDelay'])/2)
                 .sort_values(by='AvgDelay', ascending=False)
                )

org_dest_pcts = pd.merge(org_dest_pcts, delayed_by_origin_dest_count, on=['Origin','Dest'])

org_dest_pcts[org_dest_pcts['DepCount'] > 50].nlargest(15, 'AvgDelay')

,Origin,Dest,PercDepDelay,PercArrDelay,AvgDelay,DepCount
71,ASE,ORD,0.509,0.537,0.523,55
72,DFW,MDT,0.554,0.477,0.515,65
97,IAD,BHM,0.449,0.504,0.476,127
107,EWR,LIT,0.420,0.480,0.450,174
109,PHL,PDX,0.457,0.438,0.447,81
115,ORD,ASE,0.439,0.436,0.437,57
117,SBP,LAS,0.386,0.474,0.430,57
125,ATL,BGR,0.429,0.423,0.426,105
133,TVC,MQT,0.414,0.407,0.410,87
134,EWR,BQN,0.384,0.431,0.407,232


## Can you regress how delayed a flight will be before it is delayed?

## What are the most important features for this regression?

### Feature Preparation 


In [ ]:
# Variable to change to categorical
all_years_reg = all_years_reg.categorize(['UniqueCarrier','Dest'])

In [ ]:
all_years_reg[['UniqueCarrier','Dest']].compute().info()

In [ ]:
carrier = all_years_reg[['UniqueCarrier']].compute()

In [ ]:
carrier['UniqueCarrier'].unique()

# Regression of Delay

The Dask module is a solution for processing "big data," however, the it currently does not include methods for analysis, such as generalized linear models, like other big data solutions. The following will use a series of simple random sampling and kfold cross validation to find the coefficient estimates of a linear model.

#### The following features will be explore to predict if the flight will have departure delay

##### The predicted variable will be: 
* DepDelay

##### The explanatory variables:
* Month
* DayofMonth
* DayOfWeek
* DepTime
* UniqueCarrier
* Dest

##### Possible features to Create
* Plane's flight number of the day
* Plane's arrival delay of previous flight
* Plane's age

In [ ]:
#all_years_reg = all_years[['DepDelay','DayOfWeek', 'DepTime', 'Month']].dropna().sample(.0001).compute()
X = all_years_reg[['DayOfWeek', 'DepTime', 'Month']]
y = all_years_reg['DepDelay']

In [ ]:
from sklearn import linear_model
start = datetime.now()
reg = linear_model.LinearRegression(n_jobs=-1)
reg.fit(X, y)
#print(cross_val_score(reg, ArrDelay_X, ArrDelay_y, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1))
print('Coefficients: \n', reg.coef_)
print(datetime.now() - start)

In [ ]:
from sklearn import linear_model
from sklearn.model_selection import KFold

seeds = [123,456,789,101,112]

coefs = []


# Sample the entire data set as large as possible a few times. Each time has it's own cross validation sampling.

for i in range(len(seeds)):
    start = datetime.now()
    # Take a sample from all the data
    all_years_reg = all_years[['DepDelay','DayOfWeek', 'DepTime', 'Month']].dropna().sample(.0001, random_state=seeds[i]).compute()
    print(all_years_reg.info())
    print(all_years_reg.shape)
        
    ######
    # Insert a cross validation split step here
    cv = KFold(n_splits=5)
    
    ######
    
    reg = linear_model.LinearRegression(n_jobs=-1)
    ArrDelay_X = all_years_reg[['DayOfWeek', 'DepTime', 'Month']]
    ArrDelay_y = all_years_reg[['DepDelay']]
    reg.fit(ArrDelay_X, ArrDelay_y)
    #print(cross_val_score(reg, ArrDelay_X, ArrDelay_y, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1))
    print('Coefficients: \n', reg.coef_)
    coefs.append(reg.coef_)
    print("Time to sample and regress: ", datetime.now() - start)

print(coefs) # Chart this eventually

In [ ]:
coefs[1][0][0]

### Future Work

* Optimize with index key base on Data, deptarture time, and TailNum
* Use of alternative compression, such as snappy or LZ4
    * http://java-performance.info/performance-general-compression/
* Use a diffent big data approach to find a more efficient way to estimating the linear model coefficients:
    * Spark MLLib
    * Dask GLM
    * Turi/Graphlab Create

## Bibliography

* Dask Documentation, http://dask.pydata.org/en/latest/
* Distributed Optimization and Statistical Learning via the Alternating Direction Method of Multipliers, Boyd, et al http://stanford.edu/~boyd/papers/pdf/admm_distr_stats.pdf
* https://www.transtats.bts.gov/OT_Delay/OT_DelayCause1.asp
* Variable Descriptions: http://stat-computing.org/dataexpo/2009/the-data.html
* Dask example using airline data https://jcrist.github.io/dask-sklearn-part-3.html

## Appendices

### Appendix A - CSV to Parquet Conversion

### Appendix B - Benchmark Tests